###Прочитайте файлы в rdd, проведите их анализ, и расскажите, чем они отличаются (для оценки отлично достаточно чтения файла и просмотра части его содержимого и сравнения названия колонок).

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=47de544897ea46329caaf647b403b41fb63ce816352fe23b19061f763580e5d6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[2]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', f'{int(120000 / 2 /1.1)}mb')\
        .config('spark.executor.cores', 1)\
        .getOrCreate()

sc = spark.sparkContext

In [3]:
data_train = sc.textFile('./train.csv')
data_test = sc.textFile('./test.csv')

In [4]:
data_train.take(5)

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2',
 '563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2',
 '615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2']

In [5]:
data_test.take(5)

['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',
 '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0',
 '3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1',
 '4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0']

In [6]:
header_train = data_train.first()
columns_train = header_train.split(',')

print(columns_train)

['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi', 'price_range']


In [7]:
header_test = data_test.first()
columns_test = header_test.split(',')

print(columns_test)

['id', 'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi']


In [8]:
if set(columns_train) == set(columns_test):
    print("Названия столбцов в файле 1 и файле 2 совпадают")
else:
    print("Названия столбцов в файле 1 и файле 2 не совпадают")

Названия столбцов в файле 1 и файле 2 не совпадают


In [9]:
type(data_train), type(data_test)

(pyspark.rdd.RDD, pyspark.rdd.RDD)

In [10]:
set(header_train.split(',')) ^ set(header_test.split(','))

{'id', 'price_range'}

####т.е. столбец id отсутсвует в train, но есть в test. И наоборот столбец price_range присутсвует в train и отсутсвует в test

####Загрузим данные в виде табличной структуры.

In [11]:
df_train = spark.read.format("csv").option("header", "true").load("train.csv")
df_test = spark.read.format("csv").option("header", "true").load("test.csv")
stats_train = df_train.describe()
stats_test = df_test.describe()
stats_train.show(), stats_test.show()

+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|    battery_power|              blue|       clock_speed|          dual_sim|               fc|             four_g|        int_memory|             m_dep|        mobile_wt|           n_cores|               pc|         px_height|         px_width|               ram|             sc_h|             sc_w|        talk_time|          three_g|      touch_screen|              wifi|       price_range|
+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-

(None, None)

In [12]:
type(df_train), type(df_test)

(pyspark.sql.dataframe.DataFrame, pyspark.sql.dataframe.DataFrame)

In [13]:
df_train.count(), df_test.count()

(2000, 1000)

In [14]:
df_train.printSchema(), df_test.printSchema()

root
 |-- battery_power: string (nullable = true)
 |-- blue: string (nullable = true)
 |-- clock_speed: string (nullable = true)
 |-- dual_sim: string (nullable = true)
 |-- fc: string (nullable = true)
 |-- four_g: string (nullable = true)
 |-- int_memory: string (nullable = true)
 |-- m_dep: string (nullable = true)
 |-- mobile_wt: string (nullable = true)
 |-- n_cores: string (nullable = true)
 |-- pc: string (nullable = true)
 |-- px_height: string (nullable = true)
 |-- px_width: string (nullable = true)
 |-- ram: string (nullable = true)
 |-- sc_h: string (nullable = true)
 |-- sc_w: string (nullable = true)
 |-- talk_time: string (nullable = true)
 |-- three_g: string (nullable = true)
 |-- touch_screen: string (nullable = true)
 |-- wifi: string (nullable = true)
 |-- price_range: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- battery_power: string (nullable = true)
 |-- blue: string (nullable = true)
 |-- clock_speed: string (nullable = true)
 |-- dual_s

(None, None)